In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam as adam
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
# import pandas as pd
import glob
import numpy as np
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')
import sys
sys.tracebacklimit = 0

In [3]:
my_path = Path('D:\keras_png_slices_data')

In [ ]:
x_train_path, x_val_path, x_test_path = ()